[Link](https://www.kaggle.com/competitions/digit-recognizer/data) to Competition

In [1]:
#|default_exp app

In [2]:
#| export
import os
from pathlib import Path
import pandas as pd
import numpy as np
from fastai.vision.all import *
from fastai.vision.all import Image
from torchvision.utils import save_image
from fastai.callback.all import *

matplotlib.rc('image', cmap='Greys')

In [3]:
#| export
iskaggle = os.environ.get('KAGGLE_KERNEL_RUN_TYPE', '')
creds = ''

In [4]:
#| export
cred_path = Path('~/.kaggle/kaggle.json').expanduser()
if not cred_path.exists():
    cred_path.parent.mkdir(exist_ok=True)
    cred_path.write_text(creds)
    cred_path.chmod(0o600)

In [5]:
#| export
path = Path('digit-recognizer')

In [6]:
#| export
if not iskaggle and not path.exists():
    import zipfile, kaggle
    kaggle.api.competition_download_cli(str(path))
    zipfile.ZipFile(f'{path}.zip').extractall(path)

In [7]:
#| export
if iskaggle:
    path = Path('../input/digit-recognizer')
    ! pip install -q dataset

For each of the 28000 images in the test set, output a single line containing the ImageId and the digit you predict. For example, if you predict that the first image is of a 3, the second image is of a 7, and the third image is of a 8, then your submission file would look like:

`ImageId,Label`<br>
`1,3`<br>
`2,7`<br>
`3,8` <br>
`(27997 more lines)`<br>

Import Training Set

In [8]:
#| export
train_full = pd.read_csv(path/'train.csv')
test_full = pd.read_csv(path/'test.csv')

In [9]:
#| export
label = train_full.iloc[:, :1]
test_label = test_full.iloc[:, :1]

In [10]:
#| export
image_df = train_full.iloc[:, 1:]
test_image_df = train_full.iloc[:, 1:]

In [11]:
image_df.shape

(42000, 784)

In [12]:
#| export
np_image_array = image_df.values
np_test_array = test_image_df.values

In [13]:
#| export
image_tens = [tensor(x).float() for x in np_image_array]
test_image_tens = [tensor(x).float() for x in np_test_array]

In [14]:
#| export
stacked_img_tens = torch.stack(image_tens).float()/255
stacked_test_tens = torch.stack(test_image_tens).float()/255

In [15]:
#| export
stacked_label_tens = tensor(label).unsqueeze(1)
stacked_test_label_tens = tensor(label).unsqueeze(1)

In [16]:
# img = stacked_img_tens[0].reshape(28, 28)
# save_image(img, './test.png')

In [17]:
#| export
folder_path = Path('train')
if not folder_path.exists():
    folder_path.mkdir()
for i in range(stacked_img_tens.shape[0]):
    img = stacked_img_tens[i].reshape(28, 28)
    dest = (folder_path/label['label'][i].astype(str))
    dest.mkdir(exist_ok=True)
    img_path = Path(f"{dest}/{i}.png")
    if not img_path.exists():
        save_image(img, img_path)

In [20]:
folder_path = Path('test')
if not folder_path.exists():
    folder_path.mkdir()
for i in range(stacked_test_tens.shape[0]):
    img = stacked_test_tens[i].reshape(28, 28)
    dest = (folder_path/label['label'][i].astype(str))
    dest.mkdir(exist_ok=True)
    img_path = Path(f"{dest}/{i}.png")
    if not img_path.exists():
        save_image(img, img_path)

In [ ]:
#| export
dls = ImageDataLoaders.from_folder(folder_path, get_image_files(folder_path), bs=255, valid_pct=0.2, seed=42, label_func=parent_label)

In [ ]:
#| export
learn = vision_learner(dls, resnet18, loss_func=F.cross_entropy, metrics=accuracy)

In [ ]:
#| export
learn.fine_tune(3, base_lr=1e-3)

In [ ]:
#| export
def find_lr_metrics(learn):
    lr_min, lr_steep, lr_valley = learn.lr_find(suggest_funcs=(minimum, steep, valley))
    lr_min = f"{lr_min:.2e}"
    lr_steep = f"{lr_steep:.2e}"
    lr_valley = f"{lr_valley:.2e}"
    print(f"Minimum: {lr_min}, Steepest Point {lr_steep}, Valley {lr_valley}")
    return lr_min, lr_steep, lr_valley

In [ ]:
find_lr_metrics(learn)

In [ ]:
#| export
learn = vision_learner(dls, resnet18, loss_func=F.cross_entropy, metrics=accuracy).to_fp16()
learn.fit_one_cycle(3, 1e-3)

In [ ]:
find_lr_metrics(learn)

In [ ]:
learn.unfreeze()

In [ ]:
find_lr_metrics(learn)

In [ ]:
learn.fit_one_cycle(5, lr_max=slice(1e-3, 1e-2))

In [ ]:
learn.recorder.plot_loss()

In [ ]:
learn.save('model_1')

In [ ]:
path = Path('test')

In [ ]:
import nbdev
nbdev.export.nb_export('mnist_classifier.ipynb', 'app')
print('Export successful')